# Motion Coherence Experiment Analysis Flow

## Glossary of Variable Names:
* moz/MOz = Mert Özkan
* mc: motion coherence
* f: file
* dat: data
* l: line
* idx: index
* ptr: pointer
* trl: trial
* no: number (rank)
* dxn: direction
* coh: coherence
* ok: correct
* is: prefix such as isOK = is it correct?
* rt: reaction time
* t: time
* st: string
* i: integer
* stc: sentence
* w: word
* inv: inventory
* cond: condition
* sq: order/sequence
* sxn: session
* rej: rejected
* sub: subject/participant
* w: with
* wh: *prefix* which/what
* prev: previous
* pc: percent/percentage
* dr: directory

## Synchronising Psychophysical and EEG Trials

* Discard the trials in the psychophysical data which were removed in the artefact rejection.
    * There are two-fold steps of artefact rejection implemented in the Brainvision Analyzer. So, 
* Remove invalid trials and keep them logged to discard from EEG data:
    * 1s > RT(from onset) > .1
    * Invalid key press
* Mark correct and wrong trials

### Import Data from Log Files

In [2]:
def import_psyphys_mc(f_name,path):
    '''
    Feb 22, 2019
    Mert Ozkan
    Dartmouth College
    Motion Coherence
    
    Imports datasets from log files for the Motion Coherence experiment.
    Usage: trl_no, dxn, coh, isOK, key, rt = import_psyphys_mc(f_name,path)
    '''
    
    import numpy as np
    
    dat_path = '/'.join([path,f_name])
    f = open(dat_path,'r')
    
    # Data log starts 2 lines after the data pointer
    dat_ptr = '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
    read_l = f.readlines()
    for idx in range(len(read_l)):
        if dat_ptr in read_l[idx]:
            dat_start_idx = idx + 2
            break
    trl_no = []
    dxn = []
    coh = []
    isOK = []
    key = []
    rt = [] 
    for trl in read_l[dat_start_idx:]:
        trl_dat = trl.split()
        trl_no.append(int(trl_dat[0]))
        dxn.append(int(trl_dat[1]))
        coh.append(float(trl_dat[2]))
        isOK.append(int(trl_dat[3]))
        key.append(trl_dat[4])
        rt.append(float(trl_dat[5]))
    
    trl_no = np.array(trl_no)
    dxn = np.array(dxn)
    coh = np.array(coh)
    isOK = np.array(isOK)
    key = np.array(key)
    rt = np.array(rt)
    return trl_no, dxn, coh, isOK, key, rt

### Import Subject Data inventory

In [13]:
def import_subjectdatainfo_mc(path):
    '''
    Feb 22, 2019
    Mert Ozkan
    Dartmouth College
    Motion Coherence
    
    Import subject data inventory from data_inventory_mc.txt
    Usage: sub_no, sub_name, sq, bhv_ptr, eeg_ptr, trl_rej_1, trl_rej_2 = import_subjectdatainfo_mc(path)
    '''
    
    import numpy as np
    
    f = open(path,'r')
    read_l = f.readlines()
    
    sub_no = []
    sub_name = []
    sq = []
    bhv_ptr = []
    eeg_ptr = []
    trl_rej_1 = []
    trl_rej_2 = []
    for idx in range(len(read_l)):
        if '*' in read_l[idx]:
            inv_st = read_l[idx+1].split('; ')
            sub_no.append(int(inv_st[0]))
            sub_name.append(inv_st[1])
            sq.append(
                    np.array(inv_st[2].split(', ')).astype(int))
            bhv_ptr.append(inv_st[3])
            eeg_ptr.append(inv_st[4])
            trl_rej_1.append(
                    np.array(inv_st[5].split(', ')).astype(int))
            if inv_st[6][-1:] == '\n':
                inv_st[6] = inv_st[6][:-1]
            if inv_st[6] != '':
                trl_rej_2.append(
                    np.array(inv_st[6].split(', ')).astype(int))
            else:
                trl_rej_2.append(
                    np.array([]).astype(int))
    return sub_no, sub_name, sq, bhv_ptr, eeg_ptr, trl_rej_1, trl_rej_2

## Find the unique file name in a path, given two keywords
*improve this by inputting as many keywords as possible and listing all the candidate files*

In [10]:
def findfilenameindirectory_mc(path, namecontains, end_w):
    import os
    contents = os.listdir(path)
    for i in contents:
        if namecontains in i and end_w in i:
            f_name = i
            break
    return f_name

## Combine the 3  behavioural data files in the order the experiment was conducted.
*the experiment included 3 successive sessions for 3 conditions but the eeg recording was continuous*

In [25]:
def combineconditions_mc(whSub, sq, path):
    import numpy as np
    import os
    
    conds = np.array(['translational.log', 'rotational.log', 'radial.log'])
    conds = conds[sq]
    
    prev_sxn_trl_no = 0
    prev_trl_no = np.array([])
    prev_dxn = np.array([])
    prev_coh = np.array([])
    prev_isOK = np.array([])
    prev_key = np.array([])
    prev_rt = np.array([])
    
    for cond in conds:
        f_name = findfilenameindirectory_mc(path, whSub, cond)
        trl_no, dxn, coh, isOK, key, rt = import_psyphys_mc(f_name,path)
        
        trl_no += prev_sxn_trl_no
        prev_sxn_trl_no = trl_no[-1]
        
        prev_trl_no = np.concatenate((prev_trl_no,trl_no), axis=None)
        prev_dxn = np.concatenate((prev_dxn,dxn), axis=None)
        prev_coh = np.concatenate((prev_coh,coh), axis=None)
        prev_isOK = np.concatenate((prev_isOK,isOK), axis=None)
        prev_key = np.concatenate((prev_key,key), axis=None)
        prev_rt = np.concatenate((prev_rt,rt), axis=None)
        
    return prev_trl_no, prev_dxn, prev_coh, prev_isOK, prev_key, prev_rt

## Synchronise Data Files

In [129]:
import numpy as np
import os

sub_no, sub_name, sq, bhv_ptr, eeg_ptr, trl_rej_1, trl_rej_2 = import_subjectdatainfo_mc('/Users/mertozkan/Documents/Psyc161FinalProject/MotionCoherence/MC/data_inventory_mc.txt')
dat_path = '/Users/mertozkan/Documents/Psyc161FinalProject/MotionCoherence/data'

prev_dr = os.getcwd()
os.chdir(dat_path)
for n in range(len(sub_no)):
    trl_no, dxn, coh, isOK, key, rt = combineconditions_mc(bhv_ptr[n], sq[n]-1, dat_path)
    
    check_script = len(trl_no)-(len(trl_rej_1[n])+len(trl_rej_2[n]))
    pc_rej = 100*(len(trl_rej_1[n])+len(trl_rej_2[n]))/len(trl_no)
    
    dat = np.matrix([trl_no,dxn,coh,isOK,rt])
    dat = np.delete(dat,trl_rej_1[n]-1,1)
    dat = np.delete(dat,trl_rej_2[n]-1,1)
    if check_script != dat.shape[1]:
        print('The number of trials do not match! Subject Number: {}'.format(n+1))
        
    f_name = '{}_datbhv_trlmatchedweeg.csv'.format(sub_name[n])
    dat = np.transpose(dat)
    
    np.savetxt(f_name,
                 dat, delimiter = ', ',
                 header = '''
                 Behavioural Data Log:
                     All trials matched to eeg trials
                     Trial rejection according to the behavioural criteria is still needed
                 Subject Initials: {}
                 Percentage of Rejected Trials: %{:.2f}
                 
                 trl_no, dxn, coh, isOK, rt
                 '''.format(sub_name[n],pc_rej),
               fmt='%1.5e')
os.chdir(prev_dr)